In [1]:
from urllib.request import Request, urlopen
import urllib
from bs4 import BeautifulSoup
from time import sleep
import csv
import re

In [3]:
# url to scrape the songs list from
#base_url = "http://www.hindilyrics.net/lyrics/of-{}.html"

eng_url = "http://www.hindilyrics.net/lyrics/of-{}.html"

hindi_url = "https://www.hinditracks.in/{}-lyrics"

songs = ["Mere Naam Tu","Bhankas","bekhayali","Aira Gaira","Paisa",
         "Jugraafiya","Issaqbaazi","Husn Parcham",
        "Dil Royi Jaye","Mukhda Vekh ke","Chale Aana","Humne Rait pe","Hume Tumse Pyaar Kitna",
         "Manmohini","Phir Mulaaqat"]

In [4]:
hindi_lyrics = []
english_lyrics = []
for song in songs:
    eng_ly = []
    song_ = '%20'.join(x for x in song.lower().split())
    #extract English Lyrics
    req = eng_url.format(song_)
    artist_url = Request(req, headers={'User-Agent': 'Mozilla/5.0'})
    
    print("Going to url : ", req)
    try:
        html_page = urlopen(artist_url).read()
    except urllib.error.HTTPError as e:
        print('Lyrics Not Found')
        continue
    try:
        soup = BeautifulSoup(html_page, 'html.parser')
        data = soup.find('pre').text.strip()
        for lines in data.splitlines():
            eng_ly.append(lines.strip())
            print(lines.strip())
    except AttributeError as a:
        print('Lyrics Not Found')
    #print(eng_ly)
        
    print('------------------------------------------------')

    #extract Hindi Lyrics
    hin_ly = []
    song_ = '-'.join(x for x in song.lower().split())
    req = hindi_url.format(song_)
    artist_url = Request(req, headers={'User-Agent': 'Mozilla/5.0'})
    
    print("Going to url : ", req)
    try:
        html_page = urlopen(artist_url).read()
    except urllib.error.HTTPError as e:
        print('Lyrics Not Found')
        continue

    try:
        soup = BeautifulSoup(html_page, 'html.parser')
        end = False
        divs = soup.find('div',attrs={'id':'hindilyrics'})
        hin_ly.append(divs.text.strip())
        print(divs.text.strip())
        p = divs.find_next('p')
        hin_ly.append(p.text.strip())
        print(p.text.strip())
        while not end:
            p = p.find_next('p')
            print(p.text.strip())
            hin_ly.append(p.text.strip())
            if re.match('[A-Za-z0-9]',p.find_next('p').text):
                #print(hin_ly)
                end = True
    except error:
        print('Lyrics Not Found')
    if len(eng_ly)!=0 and len(hin_ly)!=0:
        hindi_lyrics.append(hin_ly)
        english_lyrics.append(eng_ly)
    else:
        continue
    print('------------------------------------------------')

print(len(hindi_lyrics))
print(len(english_lyrics))
    
    

Going to url :  http://www.hindilyrics.net/lyrics/of-mere%20naam%20tu.html
Woh rang bhi kya rang hain
Milata nah jo tere hot ke rang se hubahu
Woh khushbu kya khushbu
Thehare na jo teri saanwari zulf ke rubaru

Tere aage yeh duniya hain fiki si
Mere bin tu na hogi kisi ki bhi
Ab yeh jahir sare aam hain alaan hain
Jab tak jahan mein subah sham hain tab tak mere naam tu
Jab tak jahan mein tab tak mere naam tu

Jab tak jahan mein subah sham hain tab tak mere naam tu
Jab tak jahan mein tab tak mere naam tu

Uljhan bhi hoon teri uljhan ka hal bhi hoon mein
Thoda sa ziddi hoon thoda pagal bi hoon mein
Barkha bijali badal jhoote jhuti fulon ki sau baatein
Sachi tu hain sacha mein hoon sache apani dil ki baatein
Dastak haaton se haath pe kar de tu na kar aankho pe palko ke parde
Kya yeh itana bada kaam hain alaan hain
(Jab tak jahan mein subah sham hain tab tak mere naam tu
Jab tak jahan mein mera naam hain tab tak mere naam tu) - 2

Mere hi gheremein ghumeingi har pal tu aise
Suraj ke gheremi

NameError: name 'error' is not defined

In [39]:
hindi_lyrics[0]

['वो रंग भी क्या रंग है\nमिलता ना जो तेरे\nहोठ के रंग से हुबहू\nवो खुशबू क्या खुशबू\nठहरे ना जो तेरी सांवरी जुल्फ के रूबरू \nतेरे आगे ये दुनिया है फीकी सी\nमेरे बिन तू ना होगी किसी की भी\nअब ये ज़ाहिर सरेआम है, ऐलान है..\nजब तक जहान में सुबह शाम है\nतब तक मेरे नाम तू\nजब तक जहान में मेरा नाम है\nतब तक मेरे नाम तू \nजब तक जहान में सुबह शाम है\nतब तक मेरे नाम तू\nजब तक जहान में मेरा नाम है\nतब तक मेरे नाम तू \nउलझन भी हूँ तेरी\nउलझन का हल भी हूँ मैं\nथोड़ा सा जिद्दी हूँ\nथोड़ा पागल भी हूँ मैं \nबरखा बिजली बादल झूठे\nझूठी फूलों की सौगातें\nसच्ची तू है सच्चा मैं हूँ\nwww.hinditracks.in\nसच्ची अपने दिल की बातें \nदस्तख़त हाथों से हाथों पे कर दे तू\nना कर आँखों पे पलकों के परदे तू\nक्या ये इतना बड़ा काम है, ऐलान है \nजब तक जहान में सुबह शाम है\nतब तक मेरे नाम तू\nजब तक जहान में मेरा नाम है\nतब तक मेरे नाम तू \nजब तक जहान में सुबह शाम है\nतब तक मेरे नाम तू\nजब तक जहान में मेरा नाम है\nतब तक मेरे नाम तू \nमेरे ही घेरे में घूमेगी हर पल तू ऐसे\nसूरज के घेरे में रहती है धरती ये जैसे\nपाएगी तू खुद

In [112]:
file = open('./hindi_lyrics.txt','w')
for i in range(0,len(hindi_lyrics)):
    ly = ''
    lines = hindi_lyrics[i]
    for l in lines:
        if not l.startswith('#'):
            l = re.sub(r'[a-zA-Z0-9\.\:\,\(\)]','',l)
            file.write(l)
file.close()

वो रंग भी क्या रंग है
मिलता ना जो तेरे
होठ के रंग से हुबहू
वो खुशबू क्या खुशबू
ठहरे ना जो तेरी सांवरी जुल्फ के रूबरू 
तेरे आगे ये दुनिया है फीकी सी
मेरे बिन तू ना होगी किसी की भी
अब ये ज़ाहिर सरेआम है ऐलान है
जब तक जहान में सुबह शाम है
तब तक मेरे नाम तू
जब तक जहान में मेरा नाम है
तब तक मेरे नाम तू 
जब तक जहान में सुबह शाम है
तब तक मेरे नाम तू
जब तक जहान में मेरा नाम है
तब तक मेरे नाम तू 
उलझन भी हूँ तेरी
उलझन का हल भी हूँ मैं
थोड़ा सा जिद्दी हूँ
थोड़ा पागल भी हूँ मैं 
बरखा बिजली बादल झूठे
झूठी फूलों की सौगातें
सच्ची तू है सच्चा मैं हूँ

सच्ची अपने दिल की बातें 
दस्तख़त हाथों से हाथों पे कर दे तू
ना कर आँखों पे पलकों के परदे तू
क्या ये इतना बड़ा काम है ऐलान है 
जब तक जहान में सुबह शाम है
तब तक मेरे नाम तू
जब तक जहान में मेरा नाम है
तब तक मेरे नाम तू 
जब तक जहान में सुबह शाम है
तब तक मेरे नाम तू
जब तक जहान में मेरा नाम है
तब तक मेरे नाम तू 
मेरे ही घेरे में घूमेगी हर पल तू ऐसे
सूरज के घेरे में रहती है धरती ये जैसे
पाएगी तू खुदको ना मुझसे जुदा
तू है मेरा आधा सा हिस्सा सदा
टुकड़े कर चाहे ख

In [126]:
file = open('./english_lyrics.txt','w')
for i in range(0,len(english_lyrics)):
    for lines in english_lyrics[i]:
        lines = re.sub(r'[\.\,\(\)]','',lines.lower())
        file.writelines(lines+'\n')
file.close()

In [47]:
hindi_lyrics = '/home/stuti/Documents/Jan_Intern/GPU/GPU_Train_transliteration/train/hindi_lyrics.txt'
english_lyrics = '/home/stuti/Documents/Jan_Intern/GPU/GPU_Train_transliteration/train/english_lyrics.txt'
with open(hindi_lyrics) as file:
    txt = file.read()
    hindi_words = txt.split()
with open(english_lyrics) as file:
    txt = file.read()
    english_words = txt.split()

In [49]:
import csv
with open('./songs.csv','w') as outfile:
    writer = csv.writer(outfile,escapechar=' ',quoting=csv.QUOTE_NONE)
    writer.writerow(['English','Hindi'])
    for i in range(len(hindi_words)):
        writer.writerow([english_words[i],hindi_words[i]])